In [1]:
import getpass
import requests
import urllib.parse
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')# for TextBlob or pos
nltk.download('punkt')# for tokenize
nltk.download('averaged_perceptron_tagger') # for pos
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from textblob import TextBlob

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
d:\karan\projects\ai\venvs\nlp\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because th

## Connectivity with Bing Search

In [2]:
search_url = "https://eastus.api.cognitive.microsoft.com/bing/v7.0/search"
subscription_key = getpass.getpass()

········


In [49]:
#https://docs.microsoft.com/en-us/rest/api/cognitiveservices-bingsearch/bing-web-api-v7-reference
def bingSearch(search_term: str, lang = 'en'):
    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    #urllib.parse.quote(search_term)
    params = {"q": search_term, "textDecorations": False, "textFormat": "HTML", 'setLang': lang, 'count':50}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

In [25]:
result = bingSearch("AML")['webPages']['value']
result[0]

{'id': 'https://eastus.api.cognitive.microsoft.com/api/v7/#WebPages.0',
 'name': 'Anti Money Laundering (AML) Definition',
 'url': 'https://www.investopedia.com/terms/a/aml.asp',
 'isFamilyFriendly': True,
 'displayUrl': 'https://www.investopedia.com/terms/a/aml.asp',
 'snippet': 'Anti Money Laundering (AML) seeks to deter criminals by making it harder for them to hide ill-gotten money. Criminals use money laundering to conceal their crimes and the money derived from them....',
 'dateLastCrawled': '2021-06-11T16:13:00.0000000Z',
 'language': 'en',
 'isNavigational': False}

## AML Dicts

In [27]:
amldict = {'blacklist':0, 'sanction':0, 'ofac':0, 'embargo':0}
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [6]:
langToStopWord = {'nl':'dutch',
'en':'english',
'fr':'french',
'de':'german',
'ru':'russian',
'it':'italian',
'es':'spanish'}

## Text Preprocessing
- remove stopwords like a, an, um
- remove punctuation
- lemmatize best, better to equivalent words like good

In [7]:

exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc, langShort='english'):
    langFull = langToStopWord[langShort] if langToStopWord.__contains__(langShort) else 'english'
    stop = set(stopwords.words(langFull))
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free =''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [8]:
clean(result[0]['snippet'], result[0]['language'])

'anti money laundering aml seek deter criminal making harder hide illgotten money criminal use money laundering conceal crime money derived them'

In [9]:
clean('blacklist sanction ofac embargo')

'blacklist sanction ofac embargo'

## gensim model download

In [10]:
#!curl -o ../downloads/GoogleNews-vectors-negative300.bin.gz https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

^C


In [11]:
#!gzip -d ../downloads/GoogleNews-vectors-negative300.bin.gz

gzip: ../downloads/GoogleNews-vectors-negative300.bin already exists;	not overwritten


In [13]:
model = gensim.models.KeyedVectors.load_word2vec_format('../downloads/GoogleNews-vectors-negative300.bin', binary=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 1570M    0  146k    0     0   146k      0  3:03:24  0:00:01  3:03:23 93892
  0 1570M    0 1421k    0     0   710k      0  0:37:43  0:00:02  0:37:41  541k
  0 1570M    0 4531k    0     0  1132k      0  0:23:39  0:00:04  0:23:35 1090k
  0 1570M    0 4547k    0     0  1136k      0  0:23:34  0:00:04  0:23:30  996k
  0 1570M    0 4771k    0     0   954k      0  0:28:05  0:00:05  0:28:00  969k
  0 1570M    0 11.7M    0     0  2013k      0  0:13:18  0:00:06  0:13:12 2315k
  1 1570M    1 21.0M    0     0  3072k      0  0:08:43  0:00:07  0:08:36 3931k
  1 1570M    1 30.2M    0     0  3870k      0  0:06:55  0:00:08  0:06:47 5792k
  2 1570M    2 40.1M    0     0  4572k      0  0:05

In [14]:
model.similarity('sanction', 'penalty')#promising

0.44346452

In [15]:
model.similarity('embargo', 'ban')#promising

0.48116162

In [30]:
model.similarity('sanction', 'ban')

0.45747817

In [57]:
model.similarity('blacklist', 'list')

0.5215435

In [16]:
model.similarity('sanction', 'embargo')#now aml words are eating into each other

0.2879609

In [17]:
try:
    model.similarity('schwarze','blacklist')
except KeyError:
    print('not present')

not present


## removal of proper noun

In [ ]:
blob = TextBlob("Sanction was imposed on CCCC company")

In [18]:
print(nltk.pos_tag(nltk.word_tokenize("Sanction was imposed on CCCC company")))

[('Sanction', 'NN'), ('was', 'VBD'), ('imposed', 'VBN'), ('on', 'IN'), ('CCCC', 'NNP'), ('company', 'NN')]


In [20]:
print(nltk.pos_tag(nltk.word_tokenize("Apple was founded by Steve Jobs")))

[('Apple', 'NNP'), ('was', 'VBD'), ('founded', 'VBN'), ('by', 'IN'), ('Steve', 'NNP'), ('Jobs', 'NNP')]


In [22]:
def removeProperNoun(sentence):
    return ' '.join([word for word,tag in nltk.pos_tag(nltk.word_tokenize(sentence)) if tag != 'NNP' and tag != 'NNPS'])

In [23]:
removeProperNoun("Apple was founded by Steve Jobs")

'was founded by'

## count keywords


In [42]:
# from https://sanctionscanner.com/knowledge-base/un-sanctions-list-50
text="""
Governments fight financial crime globally with new sanctions, including lists of states, entities, or individuals suspected of engaging in or being consistent with illegal activities.

UN sanctions are diplomatic decisions enforced by the United Nations organizations and states against other states, entities, or individuals. These sanctions are measures of safety to preserve national safety interests, peace, and international law.

The international sanctions are generally special restrictions on cultural, economic, trading, and diplomatic relationships with a certain country designated individual or organization.

It is crucial to know if the client is subject to specific regulations or the corporation is coming from a sanctioned country. By knowing this information, institutions act accordingly and blocking them or applying needed restrictions to their interaction.

Businesses have to comply with these regulations. Otherwise, they will face harsh legal measurements, such as criminal and civil penalties.

Sanctions and the way of handling them are wide and depend on a reason or a specific case to applied regulations. Firms or individuals subjected to sanctions may be specific individuals prohibited from the UN, such as states or terrorists, meaning that their nationals have become influenced by restrictions like trade barriers or travel ban.

"""

def runPipeLine(text):
    cleanedText = clean(removeProperNoun(text))
    for textToken in nltk.word_tokenize(cleanedText):
        for key in amldict:
            try:
                score = 0.4
                if(textToken == 'ban'):
                    score = 0.48
                if(model.similarity(textToken,key)>score):
                    print(f'{textToken} === {key}')
                    amldict[key]=amldict[key]+1
            except KeyError:
                pass
runPipeLine(text)
print(amldict)

sanction === sanction
list === blacklist
sanction === sanction
sanction === sanction
sanction === sanction
restriction === embargo
sanctioned === sanction
restriction === embargo
penalty === sanction
sanction === sanction
sanction === sanction
restriction === embargo
ban === embargo
{'blacklist': 1, 'sanction': 8, 'ofac': 0, 'embargo': 4}


## Run the pipeline

In [58]:
def reset():
    amldict = {'blacklist':0, 'sanction':0, 'ofac':0, 'embargo':0}

In [56]:
company = 'ABU SAYYAF GROUP'
searchTerm = ' AND (blacklist OR sanction OR ofac OR embargo)'

result = bingSearch(f"{company}{searchTerm}")['webPages']['value']
for searchResult in result: #filter by lang to improve error handling
    runPipeLine(f"{searchResult['snippet']} {searchResult['name']}")

list === blacklist
list === blacklist
list === blacklist
sanction === sanction


In [55]:
print(company)
print(amldict)

ABU SAYYAF GROUP
{'blacklist': 3, 'sanction': 1, 'ofac': 0, 'embargo': 2}


In [59]:
## next exp
reset()
company = 'EMARAT KAVKAZ'
result = bingSearch(f"{company}{searchTerm}")['webPages']['value']
for searchResult in result: #filter by lang to improve error handling
    runPipeLine(f"{searchResult['snippet']} {searchResult['name']}")

sanction === sanction
list === blacklist
sanction === sanction
list === blacklist
ban === embargo
embargo === embargo
list === blacklist
sanction === sanction
list === blacklist
ban === embargo
embargo === embargo
list === blacklist
list === blacklist
list === blacklist
list === blacklist
sanction === sanction
list === blacklist
sanction === sanction
list === blacklist
ban === embargo
embargo === embargo
embargo === embargo
sanction === sanction
list === blacklist
ban === embargo
embargo === embargo
blacklist === blacklist
sanction === sanction
list === blacklist
proscription === sanction
list === blacklist
list === blacklist
proscription === sanction
list === blacklist
blacklist === blacklist
list === blacklist
sanction === sanction


In [60]:
print(company)
print(amldict)

EMARAT KAVKAZ
{'blacklist': 24, 'sanction': 12, 'ofac': 0, 'embargo': 11}
